In [1]:
import os
import sys
import json
import pandas as pd

In [4]:
# set up system paths
qspath = './qgis_sys_paths.csv' 
# provide the path where you saved this file.
paths = pd.read_csv(qspath).paths.tolist()
sys.path += paths
# set up environment variables
qepath = './qgis_env.json'
js = json.loads(open(qepath, 'r').read())
for k, v in js.items():
    os.environ[k] = v
# In special cases, we might also need to map the PROJ_LIB to handle the projections
# for mac OS
os.environ['PROJ_LIB'] = '/Applications/Qgis.app/Contents/Resources/proj'

In [5]:
import PyQt5.QtCore
import gdal
import qgis.PyQt.QtCore

ModuleNotFoundError: No module named 'PyQt5'